In [1]:
import os

from lungmask import mask
import SimpleITK as sitk
import pydicom as dicom

import cv2

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import MinMaxScaler

import skimage

In [2]:
dicomdir = 'osic-pulmonary-fibrosis-progression/train/' # set the directory the DICOM folders are stored in
patients = []

# Create a list of patient IDs from folder names
for root, dirs, files in os.walk(dicomdir):
    patients += dirs

In [3]:
# Functions to load DICOMs to numpy arrays and convert images to HUs

## https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/

def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def process_scan(scandir):
    # Load the specified scan, create an array with HUs
    slices = load_scan(scandir)
    slices_hu = get_pixels_hu(slices)
    
    #Create the mask for the lungs
    lung_mask = mask.apply(sitk.GetImageFromArray(slices_hu))
    lung_mask[lung_mask > 0] = 1
    
    #Apply the mask to the scan, black out anything not lungs with HU -1500
    masked = slices_hu.copy()
    masked[(lung_mask == 0)] = -1500
    
    # Find the borders of the lungs and then crop the empty space
    non_empty_rows = []
    non_empty_columns = []
    for i in range(masked.shape[0]):
        non_empty_rows += np.where(masked[i].max(axis=0) > -1400)[0].tolist()
        non_empty_columns += np.where(masked[i].max(axis=0) > -1400)[0].tolist()
    
    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))
    cropped = masked[:, cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1]
    
    # Clip the minimum and maximum pixel values
    clipped = np.clip(cropped, -1200, 300).astype('float32')
    
    # Scale the pixel values to be between -1 and 1

    scaled = np.zeros(clipped.shape)

    scaler = MinMaxScaler()
    scaler.set_params(feature_range=(-1,1))
    for i in range(len(clipped)):
        temp = scaler.fit_transform(clipped[i])
        scaled[i,:,:] = temp
        
    # Drop any slices that have nothing or almost nothing in them
    filtered = scaled.copy()
    todrop = []

    for i in range(len(scaled)):
        if scaled[i].sum() < (scaled.shape[1]**2 * -0.97):
            todrop.append(i)

    filtered = np.delete(filtered, todrop, axis=0)
    
    # Resize the series to 20 x 200 x 200
    resized = skimage.transform.resize(filtered, (20,200,200), anti_aliasing=True) 
    
    return resized

In [4]:
ctdf = pd.DataFrame(index=patients, columns=['ct'])

for count, i in enumerate(patients):
    print("Processing series " + str(count+1) + " of " + str(len(patients)))
    try:
        processed = process_scan(dicomdir + i)
        ctdf.loc[i, 'ct'] = processed
    except:
        print("Error processing, dropping from dataframe")
        ctdf.drop(i, axis=0, inplace=True)

Processing series 1 of 175


100%|██████████| 2/2 [00:00<00:00, 2129.63it/s]


Processing series 2 of 175


100%|██████████| 6/6 [00:00<00:00, 1167.03it/s]


Processing series 3 of 175


...

100%|██████████| 2/2 [00:00<00:00, 131.45it/s]


Processing series 174 of 175


100%|██████████| 8/8 [00:00<00:00, 59.34it/s]


Processing series 175 of 175


100%|██████████| 8/8 [00:00<00:00, 749.17it/s]


In [7]:
ctdf.to_pickle('ctdf.pkl')

In [4]:
ctdf = pd.read_pickle('ctdf.pkl')

In [7]:
for i in ctdf.index:
    resized = skimage.transform.resize(ctdf.loc[i,'ct'], (20,100,100), anti_aliasing=True)
    ctdf.loc[i,'ct'] = resized

In [8]:
ctdf.to_pickle('ctdf_100.pkl')